<a href="https://colab.research.google.com/github/jjkcoding/League-Of-Legends-Analysis/blob/main/LOL_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import time
import tqdm

num_players = 150
num_matches = 50
sleep_timer = 121

In [ ]:
api_base = "https://na1.api.riotgames.com"
api_base2 = "https://americas.api.riotgames.com"
queue_type = "RANKED_SOLO_5x5"
api_key = ""

with open('/content/lol_key.txt') as f:
  api_key = f.read()

In [ ]:
lead_url = api_base + "/lol/league/v4/challengerleagues/by-queue/" + queue_type + "?api_key=" + api_key
lead_resp = requests.get(lead_url)
lead_info = lead_resp.json()

lead_df = pd.DataFrame(lead_info['entries'])
lead_df = lead_df.sort_values(by = "leaguePoints", ascending = False).reset_index(drop = True)[:num_players]
lead_df.head()

,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,UaB4zUU8kwQYoMRJ5H9iKX09KXrEdsVZe2uKCDkfY0gx3v...,twtv Gryffinn,1852,I,288,236,True,False,False,True
1,lJLVu4QQGR9MMrzQg_a-r-yUp8QA5wRCQ0zVfbXlqOJWR9...,Iuger,1831,I,148,87,True,False,False,False
2,aH77U_5piaal7Z_bN1knJ5pllBKLqQrfACMgsjlOOBmaWpY,Draven696969,1827,I,355,292,True,False,False,True
3,TscWRn3vyXdpG2pp8SfBmc1N8Y24y5y_IAaieXc9ADsFhtw,DouyinTonyTop,1757,I,254,186,True,False,False,False
4,8Z-v54fwz2YrVS4KXVjrb70aEgIz04qSc2_YKXV4W9rlRvpx,TL Honda APA,1682,I,353,287,True,False,False,False


In [ ]:
summ_list = lead_df['summonerName']
summ_list[:5] # View head

0    twtv Gryffinn
1            Iuger
2     Draven696969
3    DouyinTonyTop
4     TL Honda APA
Name: summonerName, dtype: object

In [ ]:
full_puuid = []

i = 0
while i < len(summ_list):
  puuid_url = api_base + "/lol/summoner/v4/summoners/by-name/" + summ_list[i] + "?api_key=" + api_key
  puuid_resp = requests.get(puuid_url)
  if puuid_resp.status_code  == 404:
    full_puuid.append("")
    print("Summoner: " + summ_list[i] + " did not work")
    i += 1
  elif puuid_resp.status_code  == 429:
    print("Sleeping")
    time.sleep(sleep_timer)
    print("Continue")
  else:
    puuid_info = puuid_resp.json()
    full_puuid.append(puuid_info['puuid'])
    i += 1

lead_df['puuid'] = full_puuid
lead_df = lead_df[lead_df['puuid'] != ""]
lead_df.to_csv("top150.csv", index = False)
lead_df

Summoner: GG RoseTh0rn did not work
Summoner: Im On Main did not work
Sleeping
Continue


,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak,puuid
0,UaB4zUU8kwQYoMRJ5H9iKX09KXrEdsVZe2uKCDkfY0gx3v...,twtv Gryffinn,1852,I,288,236,True,False,False,True,79_Yke3eGSkTfNzXmIKLMhTwaS5hlhk8UoWNFXzVmauxW2...
1,lJLVu4QQGR9MMrzQg_a-r-yUp8QA5wRCQ0zVfbXlqOJWR9...,Iuger,1831,I,148,87,True,False,False,False,MDfV4_SjuO6P-izsWgzD-ooc-M7SZ9T_IlQiJ0BoJgjq3T...
2,aH77U_5piaal7Z_bN1knJ5pllBKLqQrfACMgsjlOOBmaWpY,Draven696969,1827,I,355,292,True,False,False,True,1fKEhmW3w5pmv3epq6HnxxQXrPMyVq-p3jq6szdyGDHgqQ...
3,TscWRn3vyXdpG2pp8SfBmc1N8Y24y5y_IAaieXc9ADsFhtw,DouyinTonyTop,1757,I,254,186,True,False,False,False,6YGd7_WUKmddxJIONbHLZxGegPn9yPzPoAQodzlbzGAos0...
4,8Z-v54fwz2YrVS4KXVjrb70aEgIz04qSc2_YKXV4W9rlRvpx,TL Honda APA,1682,I,353,287,True,False,False,False,9X-yshxbcqUIlbAuNLayIZ88zUvXTc6A4v6QcpWmCgDl9G...
...,...,...,...,...,...,...,...,...,...,...,...
145,d9Zy4T1wyp1iwcZ3aaAp4Xe6budwv4aARp3IY8uqY3Eo6_...,C9 Berzerker,1080,I,241,190,False,False,True,False,Fjm1xHcj-bhssU82BBp6GjiJJ1p6G3tn8n8cvEWkDPSRF1...
146,Pw6lRB-3HGSUaI_XButD8m8epXfE0ARrF98pvu-pTKDHXs...,2 TTV Karhus,1078,I,89,60,False,False,False,False,hJCsX6_G4E-QGM28IBTI_5d3i9TYAVyUb53sWiShslazSj...
147,3TZ_hVfrNmDEXzfbL2aNUPGXNKLQIjH_3J7MvxfUJrW4Dv...,Msjin,1078,I,139,79,False,False,False,False,f4KTCDYgUTvG-170Sn-8H8GvxRsmrEL1o8vJX97kdQpsXf...
148,CLmCItMIzHtKM8aYOanzj4tEyrw-gFWkZQK0oPuGaUSykj...,Pigeon Enjoyer,1078,I,150,109,False,False,False,False,SmAJeuqYsHdG8Jb70qsWP06NdG7H3kO_srLgUrT39FSTwp...


In [ ]:
lead_df = pd.read_csv("/content/top150.csv")
lead_df

,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak,puuid
0,UaB4zUU8kwQYoMRJ5H9iKX09KXrEdsVZe2uKCDkfY0gx3v...,twtv Gryffinn,1852,I,288,236,True,False,False,True,79_Yke3eGSkTfNzXmIKLMhTwaS5hlhk8UoWNFXzVmauxW2...
1,lJLVu4QQGR9MMrzQg_a-r-yUp8QA5wRCQ0zVfbXlqOJWR9...,Iuger,1831,I,148,87,True,False,False,False,MDfV4_SjuO6P-izsWgzD-ooc-M7SZ9T_IlQiJ0BoJgjq3T...
2,aH77U_5piaal7Z_bN1knJ5pllBKLqQrfACMgsjlOOBmaWpY,Draven696969,1827,I,355,292,True,False,False,True,1fKEhmW3w5pmv3epq6HnxxQXrPMyVq-p3jq6szdyGDHgqQ...
3,TscWRn3vyXdpG2pp8SfBmc1N8Y24y5y_IAaieXc9ADsFhtw,DouyinTonyTop,1757,I,254,186,True,False,False,False,6YGd7_WUKmddxJIONbHLZxGegPn9yPzPoAQodzlbzGAos0...
4,8Z-v54fwz2YrVS4KXVjrb70aEgIz04qSc2_YKXV4W9rlRvpx,TL Honda APA,1682,I,353,287,True,False,False,False,9X-yshxbcqUIlbAuNLayIZ88zUvXTc6A4v6QcpWmCgDl9G...
...,...,...,...,...,...,...,...,...,...,...,...
143,d9Zy4T1wyp1iwcZ3aaAp4Xe6budwv4aARp3IY8uqY3Eo6_...,C9 Berzerker,1080,I,241,190,False,False,True,False,Fjm1xHcj-bhssU82BBp6GjiJJ1p6G3tn8n8cvEWkDPSRF1...
144,Pw6lRB-3HGSUaI_XButD8m8epXfE0ARrF98pvu-pTKDHXs...,2 TTV Karhus,1078,I,89,60,False,False,False,False,hJCsX6_G4E-QGM28IBTI_5d3i9TYAVyUb53sWiShslazSj...
145,3TZ_hVfrNmDEXzfbL2aNUPGXNKLQIjH_3J7MvxfUJrW4Dv...,Msjin,1078,I,139,79,False,False,False,False,f4KTCDYgUTvG-170Sn-8H8GvxRsmrEL1o8vJX97kdQpsXf...
146,CLmCItMIzHtKM8aYOanzj4tEyrw-gFWkZQK0oPuGaUSykj...,Pigeon Enjoyer,1078,I,150,109,False,False,False,False,SmAJeuqYsHdG8Jb70qsWP06NdG7H3kO_srLgUrT39FSTwp...


In [ ]:
all_match_ids = []
i = 0

while i < len(lead_df['puuid']):
  match_url = api_base2 + "/lol/match/v5/matches/by-puuid/" + lead_df['puuid'][i] + "/ids?start=0&count=" + str(num_matches) + "&api_key=" + api_key
  match_resp = requests.get(match_url)
  if match_resp.status_code  == 404:
    print("Summoner: " + summ_list[i] + " did not work")
    i += 1
  elif match_resp.status_code  == 429:
    print("Sleeping")
    time.sleep(sleep_timer)
    print("Continue")
  else:
    match_info = match_resp.json()
    all_match_ids += match_info
    i += 1

all_match_ids = list(set(all_match_ids)) # Get unique match ids
with open('match_list.txt','w') as tfile:
	tfile.write('\n'.join(all_match_ids))

Sleeping
Continue


In [ ]:
with open('/content/match_list.txt') as f:
  all_match_ids = f.readlines()
  all_match_ids = list(map(lambda x: x.replace("\n", ""), all_match_ids))

print(len(all_match_ids))
print(all_match_ids)

4842
['NA1_4658308664', 'NA1_4656285730', 'NA1_4663599595', 'NA1_4671110398', 'NA1_4658627390', 'NA1_4667529297', 'NA1_4658294962', 'NA1_4666731137', 'NA1_4667965627', 'NA1_4655556349', 'NA1_4661061020', 'NA1_4662492605', 'NA1_4636326431', 'NA1_4650184629', 'NA1_4659650170', 'NA1_4647812775', 'NA1_4656063759', 'NA1_4661214595', 'NA1_4658402992', 'NA1_4669709163', 'NA1_4651607015', 'NA1_4666785357', 'NA1_4664849338', 'NA1_4670014054', 'NA1_4665358606', 'NA1_4660193325', 'NA1_4663277704', 'NA1_4667817619', 'NA1_4663446937', 'NA1_4646331365', 'NA1_4669984449', 'NA1_4658808609', 'NA1_4663903462', 'NA1_4667730482', 'NA1_4670354433', 'NA1_4669193950', 'NA1_4646705052', 'NA1_4661037106', 'NA1_4667759875', 'NA1_4663990701', 'NA1_4658360203', 'NA1_4646285528', 'NA1_4648598298', 'NA1_4648011676', 'NA1_4666469354', 'NA1_4669759031', 'NA1_4640007805', 'NA1_4669820150', 'NA1_4643666211', 'NA1_4668562401', 'NA1_4670046334', 'NA1_4664335149', 'NA1_4665127222', 'NA1_4670972960', 'NA1_4665303366', 'NA1

In [ ]:
all_player_info = pd.DataFrame()
all_team_info = pd.DataFrame()

i = 0
while i < len(all_match_ids):
  player_dat_url = api_base2 + "/lol/match/v5/matches/" + all_match_ids[i] + "?api_key=" + api_key 
  player_dat_resp = requests.get(player_dat_url)
  
  if player_dat_resp.status_code  == 404:
    print("Match " + all_match_ids[i] + " did not work")
    i += 1
  elif player_dat_resp.status_code  == 429:
    print("Sleeping")
    time.sleep(sleep_timer)
    print("Continue")
  else:
    player_dat_info = player_dat_resp.json()
    player_temp_df = pd.DataFrame(player_dat_info['info']['participants'])
    player_temp_df['gameMode'] = [player_dat_info['info']['gameMode']] * len(player_temp_df)
    all_player_info = pd.concat([all_player_info, player_temp_df])

    cur_match = player_dat_info['info']['teams']
    for cur_team in range(len(cur_match)):
      temp_team = pd.DataFrame()
      bans = list(map(lambda x: x['championId'], cur_match[cur_team]['bans']))
      temp_team['bans'] =  [" ".join(map(str, bans))]
      temp_obj = cur_match[cur_team]['objectives']
      for obj in temp_obj.keys():
        temp_team["first" + obj.capitalize()] = temp_obj[obj]['first']
        temp_team[obj + "Kills"] = temp_obj[obj]['kills']
      temp_team['gameMode'] = player_dat_info['info']['gameMode']
      temp_team['win'] = cur_match[cur_team]['win']
      all_team_info = pd.concat([all_team_info, temp_team])
    i += 1

all_player_info = all_player_info.reset_index(drop = True)
all_team_info = all_team_info.reset_index(drop = True)

all_player_info.to_csv("all_player_df.csv", index = False)
all_team_info.to_csv("all_team_df.csv", index = False)

In [ ]:
print(all_player_info.shape)
print(all_team_info.shape)

(48210, 124)
(9642, 15)


In [ ]:
all_player_info[:15]

,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,challenges,champExperience,champLevel,...,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win,gameMode
0,0.0,5.0,3.0,0.0,0.0,0.0,0.0,"{'12AssistStreakCount': 0, 'abilityUses': 209,...",11209.0,13.0,...,2.0,9.0,0.0,0.0,14.0,0.0,1.0,6.0,False,CLASSIC
1,0.0,0.0,4.0,0.0,0.0,0.0,0.0,"{'12AssistStreakCount': 0, 'abilityUses': 454,...",9740.0,12.0,...,2.0,9.0,0.0,0.0,28.0,7.0,8.0,6.0,False,CLASSIC
2,2.0,0.0,1.0,0.0,0.0,0.0,1.0,"{'12AssistStreakCount': 0, 'abilityUses': 163,...",10265.0,13.0,...,2.0,9.0,0.0,0.0,23.0,0.0,8.0,4.0,False,CLASSIC
3,0.0,3.0,3.0,0.0,0.0,0.0,0.0,"{'12AssistStreakCount': 0, 'abilityUses': 131,...",8107.0,11.0,...,0.0,9.0,0.0,0.0,8.0,0.0,1.0,7.0,False,CLASSIC
4,1.0,0.0,8.0,0.0,0.0,0.0,0.0,"{'12AssistStreakCount': 0, 'abilityUses': 118,...",6315.0,10.0,...,0.0,9.0,0.0,0.0,56.0,5.0,5.0,26.0,False,CLASSIC
5,2.0,3.0,1.0,0.0,0.0,0.0,0.0,"{'12AssistStreakCount': 0, 'abilityUses': 325,...",11527.0,14.0,...,3.0,4.0,0.0,0.0,16.0,1.0,3.0,9.0,True,CLASSIC
6,2.0,2.0,12.0,0.0,1.0,0.0,2.0,"{'12AssistStreakCount': 0, 'abilityUses': 564,...",11427.0,13.0,...,6.0,4.0,0.0,0.0,35.0,6.0,8.0,8.0,True,CLASSIC
7,0.0,9.0,8.0,24.0,0.0,0.0,2.0,"{'12AssistStreakCount': 0, 'abilityUses': 341,...",11193.0,13.0,...,4.0,4.0,0.0,0.0,27.0,4.0,3.0,14.0,True,CLASSIC
8,0.0,4.0,10.0,0.0,0.0,0.0,5.0,"{'12AssistStreakCount': 0, 'abilityUses': 159,...",11555.0,14.0,...,7.0,4.0,0.0,0.0,22.0,0.0,2.0,8.0,True,CLASSIC
9,0.0,3.0,19.0,0.0,0.0,0.0,1.0,"{'12AssistStreakCount': 0, 'abilityUses': 94, ...",8429.0,11.0,...,5.0,4.0,0.0,1.0,48.0,3.0,1.0,27.0,True,CLASSIC


In [ ]:
all_team_info[:15]

,bans,firstBaron,baronKills,firstChampion,championKills,firstDragon,dragonKills,firstInhibitor,inhibitorKills,firstRiftherald,riftHeraldKills,firstTower,towerKills,gameMode,win
0,498 122 107 69 902,False,0,False,16,True,2,False,0,True,1,True,4,CLASSIC,False
1,555 29 518 54 497,True,1,True,24,False,1,True,2,False,1,False,9,CLASSIC,True
2,141 350 107 114 121,False,0,True,13,True,1,False,0,False,1,False,1,CLASSIC,False
3,120 222 38 203 54,False,0,False,21,False,0,False,0,True,1,True,3,CLASSIC,True
4,145 9 121 91 136,False,0,False,24,True,1,False,0,False,1,False,4,CLASSIC,False
5,9 518 114 121 134,True,2,True,58,False,3,True,1,True,1,True,7,CLASSIC,True
6,518 85 -1 200 121,False,0,True,9,False,0,False,0,False,0,False,0,CLASSIC,False
7,110 107 104 120 78,False,0,False,16,True,2,False,0,True,1,True,2,CLASSIC,True
8,11 238 111 2 119,False,1,False,63,False,6,False,1,False,0,False,7,CLASSIC,True
9,895 77 35 -1 1,True,2,True,77,True,2,True,2,False,0,True,6,CLASSIC,False
